# FT Articles ID API (Headlines)

## Retrieve FT Articles ID

In [ ]:
import requests
import time
import datetime
import json
import pandas as pd
import pickle
import sqlite3
from dateutil.relativedelta import relativedelta

Second trial with FT comments section scraping
- retrieve the article ID of the ft via http://api.ft.com/content/search/v1 (with headers X-Api-Key:k_E_Y)
- retrieve corresponding livefyre comments ID via https://session-user-data.webservices.ft.com/v1/livefyre/get_lf_bootstrap?uuid=ARTICLE_ID
- retrieve comments via https://bootstrap.ft.fyre.co/bs3/ft.fyre.co/378157/LIVEFYRE_ID/init

In [ ]:
FT_API_KEY = '59cbaf20e3e06d3565778e7b8833135b219f49c7b6e4965e5c22cae5'

## THE FT HEADLINE API DOES NOT ALLOW MUCH combined topics search, thus split them :
#TOPICS = ['Russia', 'Syria','Trump', 'Crimea', 'Ukraine','Oil']
#TOPICS = ['Afd', 'Assad', 'Brexit', 'Doping', 'Gazprom']
#TOPICS = [ 'Georgia', 'IOC', 'Le+Pen', 'Merkel']
TOPICS = ['MH17', 'NATO', 'Obama', 'Putin', 'Refugees']
TOPICS = ['Mueller', 'Flynn', 'Manafort', 'Cohen']


In [ ]:
TOPICS_string = ' OR '.join(TOPICS)
TOPICS_string

In [ ]:
dates= pd.date_range('2014-01-01','2019-01-01' , freq='1M')-pd.offsets.MonthBegin(1)
dates

In [ ]:
def set_date_range(start_date):
    start_date = start_date
    start_date_string = "lastPublishDateTime:>{0}T00:00:00Z".format(start_date.strftime("%Y-%m-%d"))
    end_date = start_date + relativedelta(months=+1,days=+1)
    end_date_string = "lastPublishDateTime:<{0}T00:00:00Z".format(end_date.strftime("%Y-%m-%d"))
    string_date_range = " AND "+ start_date_string + " AND " +end_date_string
    return string_date_range

In [ ]:
def get_FT_ID(query_string,offset=0):
    print('offset :',str(offset))
    FT_ID =[]
    number_results = 0
    url = 'http://api.ft.com/content/search/v1'
    headers = {'X-Api-Key': FT_API_KEY }
    body={"queryString": query_string,"queryContext" : {"curations" : ["ARTICLES"]}, "resultContext": {"offset": offset}}
    try:
        time.sleep(.5)
        response = requests.post(url,data=json.dumps(body),headers=headers, timeout=5)
        print(response.status_code)
        data = json.loads(response.content)
        #import ipdb;ipdb.set_trace()
        
        FT_ID = [(query_string,resp['id']) for resp in data['results'][0]['results']]
        number_results = data['results'][0]['indexCount']
        print ("number_results : ",str(number_results))
    except Exception as e:
        print(e)
        print('---'*20)
        print('ERROR')
        print (e)
        
    return FT_ID, number_results

In [ ]:
def get_monthly_result(start_date):
    results = []
    date_range_month = set_date_range(start_date)
    
    query_string = TOPICS_string + date_range_month
    print(date_range_month)
    result_0, number_results= get_FT_ID(query_string,offset=0)
    results +=result_0
    
    for i in range(100,number_results,100):
        
        result,_ = get_FT_ID(query_string,offset=i)
        results +=result
    return results

In [ ]:
all_months_results = []

for date_month in dates:
    all_months_results += get_monthly_result(date_month)

In [ ]:
df = pd.DataFrame(all_months_results)
df.info()

In [ ]:
df.to_csv('FT_ID_{0}.csv'.format('_'.join(TOPICS)))

In [ ]:
uuid= df[1].tail(1).values[0]

In [ ]:
comment = retrieve_Livefyre_comment(uuid)

In [ ]:
comment

## LiveFyre API

In [ ]:
def retrieve_Livefyre_comment(ft_ID):
    comment_data =""
    url_FT_to_Livefyre = 'https://session-user-data.webservices.ft.com/v1/livefyre/get_lf_bootstrap?uuid='
    try:
        response_livefyreID = (requests.get(url_FT_to_Livefyre+str(ft_ID),timeout=5)).content
        liverfyre_comments_url =   (json.loads(response_livefyreID))['url'].replace('bootstrap.html','init').replace('https','http')
        response_livefyre_comment = requests.get(liverfyre_comments_url,timeout=5)
        comment_data = json.loads(response_livefyre_comment.content)
        
    except Exception as e:
        print("error")
    
    return (response_livefyreID,comment_data['collectionSettings']['title'],comment_data[u'headDocument'])

In [ ]:
def get_ALL_COMMENTS():
    df = pd.read_csv('ALL_FT_IDS.csv',index_col=0)
    ALL_FT_IDS = df.FT_ID.tolist()
    ALL_FT_IDS.sort()
    LIVEFYRE = []
    for i,ft_id in enumerate(ALL_FT_IDS):
        if i>=20500:

            try:
                comment_data = retrieve_Livefyre_comment(ft_id)
                LIVEFYRE.append((ft_id,comment_data[0],comment_data[1],comment_data[2]))
            except Exception as e:
                print e


            if i%500==0:

                df_checkpoint = pd.DataFrame(LIVEFYRE,columns=['FT_ID','LIVEFYRE_ID','title','comments'])
                df_checkpoint.to_csv('df_'+str(i)+'.csv', encoding='utf-8')
                print "Checkpoint at : ", i*100.0/len(ALL_FT_IDS)

    

In [ ]:
get_ALL_COMMENTS()